In [ ]:
#Lectura y guardado de datos
import scipy.io as sio
import numpy as np

archivo = sio.loadmat('ex3data1.mat')
x = archivo['X']
y = archivo['y']

for i in range (len(y)):
    if (y[i]) == 10:
        y[i] = 0

In [ ]:
#Aleatorizacion de posiciones
import random
def unico (x, a):
    if x in a:
        return False
    else:
        return True

def reordenar(azar, x, y):
    aux = []
    aux2 = []
    for i in azar:
        aux.append(x[i])
        aux2.append(y[i])
        a = i
    return(aux, aux2, a)

numerosAzar = []

#relleno el vector numerosAzar con las posiciones aleatorizadas del vector original
while (len(numerosAzar)!=len(x)):
    aux = random.randint(0, len(x)-1)
    if (unico(aux, numerosAzar) == True):
        numerosAzar.append(aux)

aux = reordenar(numerosAzar, x, y)
X = aux[0]
Y = aux[1]

#verificacion de aleatorizacion
# ultimo = aux[2]
# print (X[4999] == x[ultimo], Y[4999] == y[ultimo])

In [ ]:
#Datos de entrenamiento y testeo

#definicion de porcentaje de datos de prueba
porcentaje = 80
cant = int(porcentaje/100*len(X))

conjuntoX = []
conjuntoY = []
testX = []
testy = []
#conjunto de entrenamiento
for i in range (cant):
    conjuntoX.append(X[i])
    conjuntoY.append(Y[i])
#conjunto de prueba (lo restante del conjunto completo)
for i in range (cant, len(X)):
    testX.append(X[i])
    testy.append(Y[i])

#transformar los valores de y a vectores 
trainY = np.zeros((cant, 10))
trainY = np.array(trainY)

trainX = np.mat(conjuntoX)

conjuntoY = np.mat(conjuntoY)
for i in range(len(conjuntoY)):
    trainY[i, conjuntoY[i, 0]] = 1

In [ ]:
#Configuracion de la red

def calcular_epsilon(layers):
    return (np.sqrt(6)/(np.sqrt(layers[0]+layers[1])))

layers = [400,25,10]
epochs = 1000
alpha = 0.5
epsilon = calcular_epsilon(layers)

In [ ]:
# Visualización de los datos
import matplotlib.pyplot as plt

_, axarr = plt.subplots(20,20,figsize=(10,10))
for i in range(20):
    for j in range(20):
       axarr[i,j].imshow(x[np.random.randint(x.shape[0])].\
reshape((20,20), order = 'F'))          
       axarr[i,j].axis('off')

In [ ]:
import matplotlib.pyplot as plt 

# Función de activación
#=======================

# Sigmoidal
def sigmoid(x):
    return 1/(1+np.exp(-x))

# Derivada de la sigmoidal
def s_prime(z):
    return np.multiply(z, 1.0-z)

In [ ]:
# Red Neuronal
#==============

def fit(X, Y, w):
    # Inicialización de cada parámetro con gradiente igual a 0
	w_grad = ([np.mat(np.zeros(np.shape(w[i]))) for i in range(len(w))])  # len(w) es igual al número de capas
	m = X.shape[0]
	salida = len(layers) - 1
	h_total = np.zeros((m, layers[salida]))  # Valor predecido de todas las muestras, m*10, probabilidad
	
	for i in range(m):
		x = X[i]   #Imagen
		y = np.mat(Y[i,:]) #Etiqueta de la imagen

  		# Propagación hacia adelante
        #============================
		a = x
		a_s = []
		for j in range(len(w)):
			a = np.mat(np.append(1, a)).T
			a_s.append(a)  # Aquí se guarda el valor a de la capa L-1 anterior
			z = w[j] * a
			a = sigmoid(z)
		h_total[i,:] = a.T
		
		# Propagación hacia atras (backpropagation)
		#===========================================
		delta = a - y.T 
		w_grad[-1] += delta * a_s[-1].T  # Gradiente de la capa L-1
		# Reverso, desde la penúltima capa hasta el final de la segunda capa, excluyendo la primera y la última capa
		for j in reversed(range(1, len(w))):
			delta = np.multiply(w[j].T*delta, s_prime(a_s[j]))  # El parámetro pasado aquí es a, No z
			w_grad[j-1] += (delta[1:] * a_s[j-1].T)

	w_grad = [w_grad[i]/m for i in range(len(w))]
	J = (1 / m) * np.sum(-Y * np.log(h_total) - (np.array([[1]]) - Y) * np.log(1 - h_total))
	return {'w_grad': w_grad, 'J': J, 'h': h_total}

In [ ]:
# Inicialización de los pesos
#=============================
def init_weights(layers, epsilon):
	weights = []
	for i in range(len(layers)-1):
		w = np.random.rand(layers[i+1], layers[i]+1)
		w = w * 2 * epsilon - epsilon
		weights.append(np.mat(w))
	return weights

In [ ]:
#Entrenamiento

# Entrenamiento de la red
#========================
w = init_weights(layers, epsilon)
result = {'J': [], 'h': []}
w_s = {}
        
for i in range(epochs):
    fit_result = fit(trainX, trainY, w)
    w_grad = fit_result.get('w_grad')
    J = fit_result.get('J')
    h_current = fit_result.get('h')
    result['J'].append(J)
    result['h'].append(h_current)
    for j in range(len(w)):
        w[j] -= alpha * w_grad[j]
    if i == 0 or i == (epochs - 1):
        w_s['w_' + str(i)] = w_grad[:]

In [ ]:
# Gráfico de la Función de Costo, J
#===================================
plt.plot(result.get('J'))
plt.xlabel('Número de iteraciones (epocas)')
plt.ylabel('Función de Costo, J')
plt.show()

In [ ]:
# Guardado de los pesos obtenidos en un archivo
archivo = open("pesos.txt", "w")

for i in range (len(w)):
    fila, columna = w[i].shape
    ii=str(i+1)
    str1="=================="
    archivo.write("==================theta"+ii+"=========================\n")
    for j in range (fila):
        archivo.write("----------------------------------\n")
        jj=str(j+1)
        archivo.write("**Unidad "+jj+"**\n")
        for k in range (columna):
            kk=str(k)
            a=str(w[i][j,k])
            archivo.write("peso "+kk+": "+a+"\n")

archivo.close()

In [ ]:
#Predicción
def predict(X, w):
    a_s = []
    for i in range (len(X)):
        a = X[i]
        for j in range (len(w)):
            a = np.mat(np.append(1, a)).T
            z = w[j] * a
            a = sigmoid(z)
        a_s.append(a)
    return a_s


p = predict(testX, w)


test = 235  #posicion del vector de las testX
posicion = np.where(p[test] == np.amax(p[test]))
print("Valor a predecir:", testy[test][0])
print("=============================")
for i in range (len(p[test])):
    print("Probabilidad de",i,":",round(p[test][i,0]*100,4),"%")
print("=============================")
print("Prediccion:", posicion[0][0])

In [ ]:
#Rendimiento

def rendimiento(x, p):
    #colocando los valores predichos en un arreglo
    prediccion = []
    for i in range (len(p)):
        pos = np.where(p[i] == np.amax(p[i]))
        prediccion.append(pos[0][0])


    #matriz de confusion
    #aumentan los valores de izquierda a derecha y de arriba hacia abajo
    matriz_confusion = np.zeros((len(p[0]), len(p[0])))
    for i in range (len(testy)):
        matriz_confusion[testy[i][0]][prediccion[i]] += 1
    print("Matriz de confusión:\n")
    print(matriz_confusion)
    

    #sumas de filas y columnas
    suma_columnas = np.sum(matriz_confusion, axis = 0)
    suma_filas = np.sum(matriz_confusion, axis = 1)
    
    

    #calculo la precision, el recall y el F-measure
    precision = []
    recall = []
    F_measure = []
    for i in range (len(p[0])):
        precision.append(round(matriz_confusion[i][i]/suma_columnas[i],4))
        recall.append(round(matriz_confusion[i][i]/suma_filas[i],4))
        F_measure.append(round(2*(recall[i]*precision[i])/(recall[i]+precision[i]),4))
    


    #matriz de confusion con sklearn
    # from sklearn import metrics
    # matriz = metrics.confusion_matrix(testy, prediccion)
    # cm_display = metrics.ConfusionMatrixDisplay(confusion_matrix  = matriz, display_labels = [False, True])
    # cm_display.plot()
    # plt.show()

    return precision, recall, F_measure

#muestro los resultados
valores = rendimiento(testX, p)
print("\n\n           Precision   Recall   F-measure")
for i in range (len(valores[0])):
    print("      ",i,": ",valores[0][i],"   ", valores[1][i],"   ", valores[2][i])
print("=============================================================")

print("Promedio:  ", round(np.mean(valores[0]),4),"   ", round(np.mean(valores[1]),4),"   ", round(np.mean(valores[2]),4))

